In [2]:
#data manipulation
import math
import pandas as pd
import numpy as np

#Pytorch geometric
import torch   
import torch_geometric
from torch import nn, Tensor
from torch_geometric.data import Data
from torch.utils.data import Dataset, DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
from torch.nn import Linear, BCELoss, LSTM, Dropout
import torch_geometric.transforms as T
import torch.nn.functional as F

from torch_geometric.nn import global_mean_pool as gap,  global_max_pool as gmp, global_add_pool as gsp

#rdkit
from rdkit import Chem                      
from rdkit.Chem import GetAdjacencyMatrix       
from scipy.sparse import coo_matrix
from rdkit.Chem import AllChem
from rdkit import Chem, DataStructs

#matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#chemprop
import chemprop
from chemprop.args import TrainArgs, PredictArgs
from chemprop.train import cross_validate, run_training, make_predictions

#sklearn
import sklearn
from sklearn.metrics import classification_report, roc_auc_score

#shuffle
from random import shuffle

#GPU
import gc

## Utility Function

### Chem

In [3]:
def onek_encoding_unk(value, choices):
    """
    Creates a one-hot encoding with an extra category for uncommon values.

    :param value: The value for which the encoding should be one.
    :param choices: A list of possible values.
    :return: A one-hot encoding of the :code:`value` in a list of length :code:`len(choices) + 1`.
             If :code:`value` is not in :code:`choices`, then the final element in the encoding is -1.
    """
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1

    return encoding

In [4]:
class Featurization_parameters:
    """
    A class holding molecule featurization parameters as attributes.
    """
    def __init__(self) -> None:

        # Atom feature sizes
        self.MAX_ATOMIC_NUM = 100
        #for one-hot-encoding
        self.ATOM_FEATURES = {
            'atomic_num': list(range(self.MAX_ATOMIC_NUM)),
            'degree': [0, 1, 2, 3, 4, 5],
            'formal_charge': [-1, -2, 1, 2, 0],
            'chiral_tag': [0, 1, 2, 3],
            'num_Hs': [0, 1, 2, 3, 4],
            'hybridization': [
                Chem.rdchem.HybridizationType.SP,
                Chem.rdchem.HybridizationType.SP2,
                Chem.rdchem.HybridizationType.SP3,
                Chem.rdchem.HybridizationType.SP3D,
                Chem.rdchem.HybridizationType.SP3D2
            ],
        }

        # Distance feature sizes
        self.PATH_DISTANCE_BINS = list(range(10))
        self.THREE_D_DISTANCE_MAX = 20
        self.THREE_D_DISTANCE_STEP = 1
        self.THREE_D_DISTANCE_BINS = list(range(0, self.THREE_D_DISTANCE_MAX + 1, self.THREE_D_DISTANCE_STEP))

        # len(choices) + 1 to include room for uncommon values; + 2 at end for IsAromatic and mass
        self.ATOM_FDIM = sum(len(choices) + 1 for choices in self.ATOM_FEATURES.values()) + 2
        self.EXTRA_ATOM_FDIM = 0
        self.BOND_FDIM = 14
        self.EXTRA_BOND_FDIM = 0
        self.REACTION_MODE = None
        self.EXPLICIT_H = False
        self.REACTION = False

In [5]:
PARAMS = Featurization_parameters()

In [6]:
def atom_features(atom: Chem.rdchem.Atom, functional_groups=None):
    """
    Builds a feature vector for an atom.

    :param atom: An RDKit atom.
    :param functional_groups: A k-hot vector indicating the functional groups the atom belongs to.
    :return: A list containing the atom features.
    """
    if atom is None:
        features = [0] * PARAMS.ATOM_FDIM
    else:
        features = onek_encoding_unk(atom.GetAtomicNum() - 1, PARAMS.ATOM_FEATURES['atomic_num']) + \
            onek_encoding_unk(atom.GetTotalDegree(), PARAMS.ATOM_FEATURES['degree']) + \
            onek_encoding_unk(atom.GetFormalCharge(), PARAMS.ATOM_FEATURES['formal_charge']) + \
            onek_encoding_unk(int(atom.GetChiralTag()), PARAMS.ATOM_FEATURES['chiral_tag']) + \
            onek_encoding_unk(int(atom.GetTotalNumHs()), PARAMS.ATOM_FEATURES['num_Hs']) + \
            onek_encoding_unk(int(atom.GetHybridization()), PARAMS.ATOM_FEATURES['hybridization']) + \
            [1 if atom.GetIsAromatic() else 0] + \
            [atom.GetMass() * 0.01]  # scaled to about the same range as other features
        if functional_groups is not None:
            features += functional_groups
    return features

In [7]:
def bond_features(bond: Chem.rdchem.Bond):
    """
    Builds a feature vector for a bond.

    :param bond: An RDKit bond.
    :return: A list containing the bond features.
    """
    if bond is None:
        fbond = [1] + [0] * (PARAMS.BOND_FDIM - 1)
    else:
        bt = bond.GetBondType()
        fbond = [
            0,  # bond is not None
            bt == Chem.rdchem.BondType.SINGLE,
            bt == Chem.rdchem.BondType.DOUBLE,
            bt == Chem.rdchem.BondType.TRIPLE,
            bt == Chem.rdchem.BondType.AROMATIC,
            (bond.GetIsConjugated() if bt is not None else 0),
            (bond.IsInRing() if bt is not None else 0)
        ]
        fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))
    return fbond

In [8]:
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 2048
#a vector representation (1x2048) for molecular feature 

In [9]:
def morgan_binary_features_generator(mol,
                                     radius: int = MORGAN_RADIUS,
                                     num_bits: int = MORGAN_NUM_BITS):
    """
    Generates a binary Morgan fingerprint for a molecule.
    :param mol: A molecule (i.e., either a SMILES or an RDKit molecule).
    :param radius: Morgan fingerprint radius.
    :param num_bits: Number of bits in Morgan fingerprint.
    :return: A 1D numpy array containing the binary Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    features_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=num_bits)
    features = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features


### Data processing

In [10]:
def data_process(dataset,batch_size):
    SMILES = dataset['SMILES']
    data_list = []
    
    for smiles in SMILES:
            
        mol = Chem.MolFromSmiles(smiles)     
        mol = Chem.AddHs(mol)  

        #generate a global vector features (binary Morgan fingerprint) and convert them
        mol_feature = torch.tensor(morgan_binary_features_generator(mol))

        xs = []
        for atom in mol.GetAtoms():
            x = atom_features(atom)
            xs.append(x)
            
        x = torch.tensor(xs)
        
        edge_indices, edge_attrs = [], []
        
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
    
            e = bond_features(bond)

            edge_indices += [[i,j],[j,i]]
            edge_attrs += [e, e]
        
        edge_index = torch.tensor(edge_indices)
        edge_index = edge_index.t().to(torch.long).view(2, -1)
        edge_attr = torch.tensor(edge_attrs).view(-1, 14)
        
        y = torch.tensor(int(dataset.loc[dataset['SMILES'] == smiles,'Activity'])) #response variable y

        smi = smiles

        # add smiles and num_feature as the attributes
        data = Data(x=x, y=y, edge_index=edge_index, edge_attr = edge_attr, smiles=smi, mol_feature=mol_feature)  
        data_list.append(data)   # store processed data into the list
        
    return DataLoader(data_list,batch_size,shuffle=True)

### Training

In [11]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [12]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

In [13]:
def train(epoch,train_loader):
    
    model.train()   
    running_loss = 0 
    correct = 0
    total = 0
    criterion = BCELoss()
    for batch in train_loader:

        optimizer.zero_grad()
        outputs = model(batch)
        label = batch.y.view(-1,1)
        loss = criterion(outputs.float(),label.float())
        

        loss.backward()   # Compute the gradient of loss function 
        optimizer.step()  # Update parameters based on gradients.
        running_loss += loss.item()
        # probability that is larger than 0.5, classify as 1 

        pred = (outputs >= 0.5).float()

        total += label.size(0)
        correct += (pred == label).float().sum()
        
    
    loss = running_loss/len(train_loader)
    accuracy = 100*correct/total
    
    train_accuracy.append(accuracy)
    train_loss.append(loss)
    
    if epoch % 10 == 0:
        print('Epoch: '+str(int(epoch)))
        print('Train Loss: %.3f | Accuracy: %.3f'%(loss,accuracy))

In [14]:
def test(epoch,test_loader):
    model.eval()
    
    running_loss = 0
    correct = 0
    total = 0
    
    
    with torch.no_grad():
        criterion = BCELoss()
        for batch in test_loader:
        
            outputs = model(batch)
            label = batch.y.view(-1,1)

            loss = criterion(outputs.float(), label.float())    
            running_loss += loss.item()
            # probability that is larger than 0.5, classify as 1 
            pred = (outputs >= 0.5).float()

            total += label.size(0)
            correct += (pred == label).float().sum()
    
        loss = running_loss/len(test_loader)
        accuracy = 100*correct/total
    
        test_accuracy.append(accuracy)
        test_loss.append(loss)
        if epoch % 10 == 0:
            print('Test Loss: %.3f | Accuracy: %.3f'%(loss,accuracy))

In [15]:
#test_set as a whole loader
def test_metrics(test_loader):
    model.eval()

    with torch.no_grad():
        labels = []
        preds = []
        for batch in test_loader:
            
            labels += list(batch.y.view(-1,1).numpy())
            preds += list(model(batch).detach().numpy())
        
        pred_labels = [1 if i > 0.5 else 0 for i in preds]
        auc = roc_auc_score(list(labels), list(preds), average='weighted')
        report = classification_report(labels, pred_labels,output_dict=True)
        return auc, report
    

In [16]:
def print_metrics(metrics):
    AUC = [] 
    precision = []
    recall = []
    f1_score = []
    accuracy = []
    for i in metrics:
        AUC.append(i[0])
        precision.append(i[1]['weighted avg']['precision'])
        recall.append(i[1]['weighted avg']['recall'])
        f1_score.append(i[1]['weighted avg']['f1-score'])
        accuracy.append(i[1]['accuracy'])
    
    print('AUC:',np.mean(AUC),'+/-',np.std(AUC))
    print('Accuracy:',np.mean(accuracy),'+/-',np.std(accuracy))
    print('Precision:',np.mean(precision),'+/-',np.std(precision))
    print('Recall:',np.mean(recall),'+/-',np.std(recall))
    print('F1-score:',np.mean(f1_score),'+/-',np.std(f1_score))
    

## Model construction

#### 1. GAT layer to update node(atom) feature vector of a graph(modelcue).
#### 2. Aggregate the updated node feature vector to capture global property
####     i.e. apply global_mean_pool function over the node features 
#### 3. Concatnate the aggregated local features with binary morgan fingerprint. This vector will be the predictors of the model. 
#### 4. Then pass the processed features to a fully connected layer for binary classification. 

In [19]:
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
    
        #before the Attention Mechanism, parse in a fully connected network and output a 50 dimensional vector 
        self.hidden = 50  
        self.in_head = 5 #repeat the mechanism for 5 times
        self.conv1 = GATConv(in_channels = 133, 
                             out_channels = self.hidden, 
                             heads=self.in_head, concat=False) #set concat to be False, so it will take average instead

        
        #fully connected layers
        self.linear1 = Linear(self.hidden+2048,100)
        self.linear2 = Linear(100, 25)
        self.linear3 = Linear(25, 10)
        self.linear4 = Linear(10, 1)

    def forward(self, data):
        
        x, edge_index, batch_index, mol_feature = data.x, data.edge_index, data.batch, data.mol_feature    
        #extract node vectors, edge_index, batch index, and binary morgan fingerprint 
        
        x = self.conv1(x.float(), edge_index)

        #aggregate the learned local node feature to capture the global property 
        #here, we apply global_mean_pool over the updated node feature
        x = gap(x,batch_index)
        #also include some other global information i.e. binary morgan fingerprint
        x = torch.cat([x, mol_feature.reshape(data.num_graphs,2048)],dim=1)

        x = F.relu(self.linear1(x.float()))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = torch.sigmoid(self.linear4(x))
        
        return x

## Model training and evaluation

### GAT model training and evalutation

In [20]:
train_losses = []
train_acc = []
test_acc = []
test_losses= []
metrics = []

for i in range(5):
    
    print('Split '+str(i+1)+' ......')
    train_set = pd.read_csv('dir_to_dataset/5_fold_cv_train_test/train_split'+str(i+1)+'.csv')
    test_set = pd.read_csv('dir_to_dataset/5_fold_cv_train_test/test_split'+str(i+1)+'.csv')
    train_loader = data_process(train_set,32)
    test_loader = data_process(test_set,len(test_set))
    
    model = GAT().float()
    model.apply(reset_weights)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
    
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    #update the model parameters with 50 epochs
    for epoch in range(100):
        train(epoch,train_loader)
        test(epoch,test_loader)
        
        early_stopping(test_loss[-1],model)
        
        if early_stopping.early_stop:
            print('Early stopping')
            break
    
    model.load_state_dict(torch.load('checkpoint.pt'))
    auc, report = test_metrics(test_loader)
    metric = [auc,report]
    
    
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    metrics.append(metric)
    
    


Split 1 ......
Reset trainable parameters of layer = Linear(133, 250, bias=False)
Reset trainable parameters of layer = GATConv(133, 50, heads=5)
Reset trainable parameters of layer = Linear(in_features=2098, out_features=100, bias=True)
Reset trainable parameters of layer = Linear(in_features=100, out_features=25, bias=True)
Reset trainable parameters of layer = Linear(in_features=25, out_features=10, bias=True)
Reset trainable parameters of layer = Linear(in_features=10, out_features=1, bias=True)
Epoch: 0
Train Loss: 0.657 | Accuracy: 77.778
Test Loss: 0.590 | Accuracy: 83.333
Validation loss decreased (inf --> 0.590289).  Saving model ...
Validation loss decreased (0.590289 --> 0.413534).  Saving model ...
Validation loss decreased (0.413534 --> 0.339591).  Saving model ...
Validation loss decreased (0.339591 --> 0.291066).  Saving model ...
Validation loss decreased (0.291066 --> 0.267089).  Saving model ...
Validation loss decreased (0.267089 --> 0.255500).  Saving model ...
Vali

In [22]:
print_metrics(metrics)

AUC: 0.903888888888889 +/- 0.018666739004489494
Accuracy: 0.9152777777777776 +/- 0.012729376930432882
Precision: 0.9149649980278163 +/- 0.013684068422941646
Recall: 0.9152777777777776 +/- 0.012729376930432882
F1-score: 0.9081744330260207 +/- 0.015409030104916148


### Chemprop training and evaluation

In [23]:
#test_set as a whole loader
def Chemprop_test_metrics(preds,labels):
    pred_labels = [1 if i[0] > 0.5 else 0 for i in preds]
    auc = roc_auc_score(list(labels), list(preds), average='weighted')
    report = classification_report(labels, pred_labels,output_dict=True)
    return auc, report

In [24]:
metrics = []
for i in range(5):
    train_args = [
        '--data_path', 'dir_to_dataset/5_fold_cv_train_test/train_split'+str(i+1)+'.csv',
        '--epochs', '100',
        '--features_generator', 'morgan',
        '--dataset_type', 'classification',
        '--save_dir', 'save_dir_to_folder_for_trained_model/model'+str(i+1),
        '--num_folds', '1',
        '--batch_size','32',
        '--metric', 'binary_cross_entropy',
        '--separate_val_path','dir_to_dataset/5_fold_cv_train_test/test_split'+str(i+1)+'.csv'
        
    ]

    mean_score, std_score = cross_validate(args=TrainArgs().parse_args(train_args), train_func=run_training)
    
    test_args = [
    '--test_path', 'dir_to_dataset/5_fold_cv_train_test/test_split'+str(i+1)+'.csv',
    '--preds_path', 'dir_to_a_csv_file_to_store_the_predictions',
    '--checkpoint_dir', 'save_dir_to_folder_for_trained_model/model'+str(i+1),
    '--num_workers', '0',
    '--features_generator','morgan'
    ]

    preds = make_predictions(args=PredictArgs().parse_args(test_args))
    test_set = pd.read_csv('dir_to_dataset/5_fold_cv_train_test/test_split'+str(i+1)+'.csv')
    labels = list(test_set['Activity'])
    
    auc, report = Chemprop_test_metrics(preds,labels)
    metric = [auc,report]
    metrics.append(metric)

Command line
python C:\Users\jimmy\anaconda3\lib\site-packages\ipykernel_launcher.py -f C:\Users\jimmy\AppData\Roaming\jupyter\runtime\kernel-2ba4bdcf-6292-4530-bc94-9d1f2a48b1e6.json
Args
{'activation': 'ReLU',
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 32,
 'bias': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': True,
 'data_path': 'C:/Users/jimmy/Desktop/FYP/train_split1.csv',
 'data_weights_path': None,
 'dataset_type': 'classification',
 'depth': 3,
 'device': device(type='cuda

 60%|█████████████████████████████████████████████████▊                                 | 9/15 [00:00<00:00, 20.72it/s]Loss = 1.8907e-03, PNorm = 37.5366, GNorm = 0.0129, lr_0 = 8.6270e-04

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 22.35it/s]
                                                                                                                       
  0%|                                                                                            | 0/5 [00:00<?, ?it/s]
                                                                                                                       Validation binary_cross_entropy = 0.367795
  8%|██████▌                                                                           | 8/100 [00:11<01:28,  1.04it/s]Epoch 8

 20%|████████████████▌                                                                  | 3/15 [00:00<00:00, 25.53it/s]Loss = 1.3878e-03, PNorm = 37.5387, GNorm

 73%|████████████████████████████████████████████████████████████▏                     | 11/15 [00:00<00:00, 23.62it/s]Loss = 5.0137e-04, PNorm = 37.6190, GNorm = 0.0129, lr_0 = 6.5184e-04

 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:00<00:00, 23.57it/s]
                                                                                                                       
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 43.68it/s]
                                                                                                                       Validation binary_cross_entropy = 0.403368
 19%|███████████████▍                                                                 | 19/100 [00:20<01:08,  1.19it/s]Epoch 19

 40%|█████████████████████████████████▏                                                 | 6/15 [00:00<00:00, 25.91it/s]Loss = 4.7675e-04, PNorm = 37.6238, GNor

 80%|█████████████████████████████████████████████████████████████████▌                | 12/15 [00:00<00:00, 22.93it/s]Loss = 2.5007e-04, PNorm = 37.6771, GNorm = 0.0055, lr_0 = 4.9252e-04

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 24.72it/s]
                                                                                                                       
 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:00<00:00, 39.44it/s]
                                                                                                                       Validation binary_cross_entropy = 0.419398
 30%|████████████████████████▎                                                        | 30/100 [00:29<00:56,  1.23it/s]Epoch 30

 53%|████████████████████████████████████████████▎                                      | 8/15 [00:00<00:00, 21.23it/s]Loss = 2.4145e-04, PNorm = 37.6795, GNor

 87%|███████████████████████████████████████████████████████████████████████           | 13/15 [00:00<00:00, 18.48it/s]
                                                                                                                       
 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:00<00:00, 35.88it/s]
                                                                                                                       Validation binary_cross_entropy = 0.425805
 41%|█████████████████████████████████▏                                               | 41/100 [00:38<00:49,  1.18it/s]Epoch 41

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]Loss = 9.7261e-05, PNorm = 37.7280, GNorm = 0.0017, lr_0 = 3.7213e-04

 60%|█████████████████████████████████████████████████▊                                 | 9/15 [00:00<00:00, 25.10it/s]Loss = 1.7035e-04, PNorm = 37.7306, GNor

...

Validating SMILES
Test size = 144
Predicting with an ensemble of 1 models


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Moving model to cuda



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.54it/s]

Saving predictions to C:/Users/jimmy/Desktop/FYP/pred_5.csv
Elapsed time = 0:00:01


In [25]:
print_metrics(metrics)

AUC: 0.8588888888888888 +/- 0.0090879405576977
Accuracy: 0.8972222222222224 +/- 0.018320702719823525
Precision: 0.8909415098812576 +/- 0.020705757230370398
Recall: 0.8972222222222224 +/- 0.018320702719823525
F1-score: 0.8902956281307205 +/- 0.020602541083328717
